# concertina button finder

## MUPHIN coding music - 2015, 2018
#### find the best path for playing any tune on a concertina

Well… here is a tune. Get your concertina. And play it. But how? Where? What keys? It is quite hard to start playing the concertina. Especially considering that quite a few notes are on several positions.

This is a tune in abc format:
<PRE>X: 1
T: Mrs Saggs
R: reel
M: 4/4
L: 1/8
K: Gmaj
| : DGA | B4 A2 GF | G6 Bd | e/2f/2gdg cgdg | g2 fe fdef |
g2 d2 cBAG | D2 C2 B,2 D2 | ECEG BAGF | G4 z  : |
| : GAB | c4 B2 AG | DFAd fdef | g2 d2 cBAG | B2 AG A2 AB |
c4 B2 AG | FA df a2 gf | g2 d2 cBAG | G4 z  : |
</PRE>


## Links
* Get your favorite tune in an abc file anywhere, for example from [thesession.org](https://thesession.org/)
* What is a concertina? Try this video of [Edel Fox at youtube](https://www.youtube.com/watch?v=axi6TrtNd_4)





In [25]:
#####################
# TODO, KNOWN BUGS  #
#####################
#  1 solved
#  2 solved
#  3 visualisation of the results   
#  4 convert all into ios/android app
#  5 solved
#  6 solved
#  7 smarter cleanup
#  8 tree instead of array with too much data
#  9 To sharpen a note precede it with the circumflex or caret ^ :: ^c
# 10 To flatten a note precede it with an underscore _ :: _B
# 11 Double sharps are shown as ^^ and double flats as __ ::
# 12 To naturalise (?) a note precede it with an equals sign = :: =c



In [26]:
# Let's start with some constant values. 
# A route is a one of the many ways to play the tune.
# $Maxroutes is the maximum number of routes this script holds in memory
# while searching for the best route. The higher, the longer calculation time.
# in general, there is no difference in results for lower values.
# tested range is 1024 ~ 16384
# Default: 4096
$Maxroutes = 4096

# true = do cleanup to the maximum defined above.
# false = no cleanup. Thats for testing only.
$cleanup=  true

# If a tune has the same note twice like A3 G3 G3 A3
# if true: script list A3 G3 A3, false: A3 G3 G3 A3
# true = better performance, less memory needed
$dupchek=  true


true

# Reading ABC
## Some basic Ruby code to get the ABC file into a hash.


In [27]:
tune={}

# READ FILE
# File.open("kesh.abc", "rb") do |f|
  File.open("mrs-saggs.abc", "rb") do |f|
# File.open("patsygearys.abc", "rb") do |f|
# File.open("joebanes-1.abc", "rb") do |f|   
# File.open("edwardtheseventh-1.abc", "rb") do |f|  
  f.each_line do |line|
    puts line
    
    line = line.chomp  # remove \n and/or \r
    if line[1]==':' && line[0]!= '|'
      line.delete!(" ")
      infotype=line[0]      
      line=line[2..999]
      tune[:number] = line if infotype=='X'
      tune[:title]  = line if infotype=='T'
      tune[:rhythm] = line if infotype=='R'
      tune[:meas]   = line if infotype=='M'
      tune[:notelen]= line if infotype=='L'
      tune[:key]    = line if infotype=='K'
      
    else 
      # this must be a line of notes  
      if tune.include?(:line)
        tune[:line] << line
      else
        tune[:line] = line
      end
      
    end # if/else
  end   # each
end     # file






X: 1

T: Mrs Saggs

R: reel

M: 4/4

L: 1/8

K: Gmaj

|: DGA | B4 A2 GF | G6 Bd | e/2f/2gdg cgdg | g2 fe fdef |

g2 d2 cBAG | D2 C2 B,2 D2 | ECEG BAGF | G4 z :|

|: GAB | c4 B2 AG | DFAd fdef | g2 d2 cBAG | B2 AG A2 AB |

c4 B2 AG | FA df a2 gf | g2 d2 cBAG | G4 z :|





#<File:mrs-saggs.abc (closed)>

# Theory and Algoritm
## Storing the Paths
Well, I am very sure there must be smarter ways (like trees) to store the paths, but I like to work with plain arrays and hashes.

Let's say we want to play A->G on the third octave,
If the A3 (note A on third octave) is on button 1, 2 and 3,
and G3 is on button 4 and 5:
there are 6 possible paths to play A3->G3:

<pre>[1,4] [1,5] [2,4] [2,5] [3,4] [3,5]</pre>

where [1,4] is playing first button 1, second button 4. You might already notice that this will grow quickly if searching a complete tune.

## Distance Matrix
If [1,4] is an easy step, and [1,5] is not, we prefer the [1,4].  We will need kind of a score to compare both options, and therefore we calculate a so called Distance Matrix.  It describes the level of difficulty to play note on button y after note on button x.

The difficulty of playing note y after note x depends on:

* stay on the same side (left or right): same side is a tiny little bit easier,
* stay on the same row: (c, g or top row): I don’t care, no score
* samefinger, same side = hopping: severe penalty
* prefer g and c row: g and c row preferred over top row
* prefer finger: playing on the inside button is good. Therefore a benefit if it’s the first (or second) button on a row
* switch from push to pull or vice versa:  a small penalty
* same note? than play it on the same button.

# Sharps

The ABC notation says F but (for example) it should be F# when playing in G Maj. This code adds the #'s. 

* The key of the tune is in tune[:key]

* sharps['F'] has all the keys where F should be F#, so sharpes note by checking 
<pre>
    sharps['F'].include? tune[:key]
</pre>



### Now update the tuneline with sharp-symbol# added 

In [28]:
sharps = {
  'F' => [   #~   F will be F# by default WHEN PLAYING IN...: 
    "G", "GMaj", "Gmaj", "GM",
    "D", "DMaj", "Dmaj", "DM",
    "A", "AMaj", "Amaj", "AM",
    "Em","Emin",
    "Bm","Bmin",
    "Cm","Cmin"
    ],
  'C' => [  #~   C will be C# by default WHEN PLAYING IN...:
    "D", "DMaj", "Dmaj", "DM",
    "A", "AMaj", "Amaj", "AM",
    "E", "EMaj", "Emaj", "EM",
    "Bm","Bmin"
    ],
  'G' => [  #~   G will be G# by default WHEN PLAYING IN...:
    "E", "EMaj", "Emaj", "EM",
    "A", "AMaj", "Amaj", "AM",
    ],
   }

{"F"=>["G", "GMaj", "Gmaj", "GM", "D", "DMaj", "Dmaj", "DM", "A", "AMaj", "Amaj", "AM", "Em", "Emin", "Bm", "Bmin", "Cm", "Cmin"], "C"=>["D", "DMaj", "Dmaj", "DM", "A", "AMaj", "Amaj", "AM", "E", "EMaj", "Emaj", "EM", "Bm", "Bmin"], "G"=>["E", "EMaj", "Emaj", "EM", "A", "AMaj", "Amaj", "AM"]}

# Parsing the Note-line into notes

In tune[:line] are the notes of the tune.  

Now delete all characters that are not notes:
* no numbers 
* no |:  :| 
* no | 
* no ^ -> make sharp
* no _ -> make flat
* other chars like / of [

In [29]:
# these strings/chars will be deleted without further notice:
deleteme = ["|:",":|",":","|]","[|", "0","1","2","3","4","6","8", "::", "|", "/", "~" , "=" ]

deleteme.each { |del|
  tune[:line].delete! del
  } 
puts tune[:line].delete! " "
listofnotes = tune[:line]

DGABAGFGBdefgdgcgdggfefdefgdcBAGDCB,DECEGBAGFGzGABcBAGDFAdfdefgdcBAGBAGAABcBAGFAdfagfgdcBAGGz


"DGABAGFGBdefgdgcgdggfefdefgdcBAGDCB,DECEGBAGFGzGABcBAGDFAdfdefgdcBAGBAGAABcBAGFAdfagfgdcBAGGz"

## now tune[:line] should contain all notes, and nothing else. 
### Any other character left will give an error while parsing, but it will not crash the script. 
### Now one step left before parsing: 
### prepare the conversion table ABC notation to  note/octave notation of this script (Ac\f\ -> A3,C4,F4)

In [30]:
# CONVERT ABC NOTE LIKE c' to C4
# TODO Key to Sharps.Flats
abc2parse = { 
"C,"=> "C1",
"D,"=> "D1",
"E,"=> "E1",
"F,"=> "F1",
"G,"=> "G1",
"A,"=> "A1",
"B,"=> "B1",
"C"=> "C2",
"D"=> "D2",
"E"=> "E2",
"F"=> "F2",
"G"=> "G2",
"A"=> "A2",
"B"=> "B2",
"c"=> "C3",
"d"=> "D3",
"e"=> "E3",
"f"=> "F3",
"g"=> "G3",
"a"=> "A3",
"b"=> "B3",
"c\'"=> "C4",
"d\'"=> "D4",
"e\'"=> "E4",
"f\'"=> "F4",
"g\'"=> "G4",
"a\'"=> "A4",
"b\'"=> "B4"
} 


# Now add the sharps 
# later version will also add any flats

abc2js={}
abc2parse.each {|key,value|
  
  note = key[0]
  if sharps[note.upcase]!= nil
    if sharps[note.upcase].include? tune[:key]
       abc2js[key]= value[0]+"#"+value[1]    # or .insert(1,"#")
    else 
       abc2js[key]= value
    end
  else 
    abc2js[key]= value
  end
  }

# thanks for keeping the results
abc2parse=abc2js.dup

{"C,"=>"C1", "D,"=>"D1", "E,"=>"E1", "F,"=>"F#1", "G,"=>"G1", "A,"=>"A1", "B,"=>"B1", "C"=>"C2", "D"=>"D2", "E"=>"E2", "F"=>"F#2", "G"=>"G2", "A"=>"A2", "B"=>"B2", "c"=>"C3", "d"=>"D3", "e"=>"E3", "f"=>"F#3", "g"=>"G3", "a"=>"A3", "b"=>"B3", "c'"=>"C4", "d'"=>"D4", "e'"=>"E4", "f'"=>"F#4", "g'"=>"G4", "a'"=>"A4", "b'"=>"B4"}

## Now lets make an array of the notes to play, notes will be in the notation of this script, like [A3 B3 C4]. 


#### this is very basic parsing

Start at the very beginning of  listofnotes
* try 3 chars, does it exist in the list of notes? do it!
* try 2 chars, does it exist? do it!
* try 1 char , does it exist? do it!
* Otherwise: error




In [31]:
tune

{:line=>"DGABAGFGBdefgdgcgdggfefdefgdcBAGDCB,DECEGBAGFGzGABcBAGDFAdfdefgdcBAGBAGAABcBAGFAdfagfgdcBAGGz", :number=>"1", :title=>"MrsSaggs", :rhythm=>"reel", :meas=>"4/4", :notelen=>"1/8", :key=>"Gmaj"}

In [32]:
a =[]

while listofnotes != nil do
  found = false
  for i in [2,1,0] do  #it should finish when found
    if not found
      if abc2parse.include? listofnotes[0..i]
        found = true 
        foundnote = listofnotes[0..i]
        
        a << abc2parse[foundnote]
        listofnotes = listofnotes[i+1..-1]    # strip i chars, keep the rest ; -1=last char of string    
      end # if include?
    end   # if notfound
  end     # for
  
  if not found    #still not found?
    puts "Thats an error on "+listofnotes[0..2]
    listofnotes=listofnotes[1..-1]
  end # if
end   # while

puts a 


Thats an error on zGA
Thats an error on z
Thats an error on 
["D2", "G2", "A2", "B2", "A2", "G2", "F#2", "G2", "B2", "D3", "E3", "F#3", "G3", "D3", "G3", "C3", "G3", "D3", "G3", "G3", "F#3", "E3", "F#3", "D3", "E3", "F#3", "G3", "D3", "C3", "B2", "A2", "G2", "D2", "C2", "B1", "D2", "E2", "C2", "E2", "G2", "B2", "A2", "G2", "F#2", "G2", "G2", "A2", "B2", "C3", "B2", "A2", "G2", "D2", "F#2", "A2", "D3", "F#3", "D3", "E3", "F#3", "G3", "D3", "C3", "B2", "A2", "G2", "B2", "A2", "G2", "A2", "A2", "B2", "C3", "B2", "A2", "G2", "F#2", "A2", "D3", "F#3", "A3", "G3", "F#3", "G3", "D3", "C3", "B2", "A2", "G2", "G2"]


## Array a is an array of notes. 

## At this point two seperate scripts are glued together. We'll do the Easy way now: 

In [33]:
Saved_old_tune= tune
tune = a

<main>:1: warning: already initialized constant Saved_old_tune
<main>:1: warning: previous definition of Saved_old_tune was here


["D2", "G2", "A2", "B2", "A2", "G2", "F#2", "G2", "B2", "D3", "E3", "F#3", "G3", "D3", "G3", "C3", "G3", "D3", "G3", "G3", "F#3", "E3", "F#3", "D3", "E3", "F#3", "G3", "D3", "C3", "B2", "A2", "G2", "D2", "C2", "B1", "D2", "E2", "C2", "E2", "G2", "B2", "A2", "G2", "F#2", "G2", "G2", "A2", "B2", "C3", "B2", "A2", "G2", "D2", "F#2", "A2", "D3", "F#3", "D3", "E3", "F#3", "G3", "D3", "C3", "B2", "A2", "G2", "B2", "A2", "G2", "A2", "A2", "B2", "C3", "B2", "A2", "G2", "F#2", "A2", "D3", "F#3", "A3", "G3", "F#3", "G3", "D3", "C3", "B2", "A2", "G2", "G2"]

# let's make a few lists

$tinakeys: the 30 buttons make 60 notes! These are mine, it can be different on yours.

In [34]:
#
# these are the concertina keys in a hash
# made in: map3.xlsx, dropbox folder 60
#

$tinakeys={
 1=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "E1"},
 2=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "A1"},
 3=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "C#2"},
 4=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "A2"},
 5=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "G#2"},
 6=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "C1"},
 7=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "G1"},
 8=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "C2"},
 9=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "E2"},
10=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "G2"},
11=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "B1"},
12=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "D2"},
13=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "G2"},
14=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "B2"},
15=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "D3"},
16=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "F1"},
17=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "BB1"},
18=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "D#2"},
19=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "G2"},
20=>{"side" => 1, "sidename" => "left", "row" => 1, "rowname" => "t", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "BB2"},
21=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "G1"},
22=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "B1"},
23=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "D2"},
24=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "F2"},
25=>{"side" => 1, "sidename" => "left", "row" => 2, "rowname" => "c", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "A2"},
26=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "A1"},
27=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "F#2"},
28=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "A2"},
29=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "C3"},
30=>{"side" => 1, "sidename" => "left", "row" => 3, "rowname" => "g", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "E3"},
31=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "C#3"},
32=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "C#3"},
33=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "G#3"},
34=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "C#4"},
35=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "A4"},
36=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "C3"},
37=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "E3"},
38=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "G3"},
39=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "C4"},
40=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "E4"},
41=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 1, "pushpull" => 1, "ppname" => "push", "note" => "G3"},
42=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 2, "pushpull" => 1, "ppname" => "push", "note" => "B3"},
43=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 3, "pushpull" => 1, "ppname" => "push", "note" => "D4"},
44=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 4, "pushpull" => 1, "ppname" => "push", "note" => "G4"},
45=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 5, "pushpull" => 1, "ppname" => "push", "note" => "D#4"},
46=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "C#3"},
47=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "D#3"},
48=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "G3"},
49=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "BB3"},
50=>{"side" => 2, "sidename" => "right", "row" => 1, "rowname" => "t", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "D4"},
51=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "B2"},
52=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "D3"},
53=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "F3"},
54=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "A3"},
55=>{"side" => 2, "sidename" => "right", "row" => 2, "rowname" => "c", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "B3"},
56=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 1, "pushpull" => 2, "ppname" => "pull", "note" => "F#3"},
57=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 2, "pushpull" => 2, "ppname" => "pull", "note" => "A3"},
58=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 3, "pushpull" => 2, "ppname" => "pull", "note" => "C4"},
59=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 4, "pushpull" => 2, "ppname" => "pull", "note" => "E4"},
60=>{"side" => 2, "sidename" => "right", "row" => 3, "rowname" => "g", "button" => 5, "pushpull" => 2, "ppname" => "pull", "note" => "F#4"}
};



{1=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>5, "pushpull"=>1, "ppname"=>"push", "note"=>"E1"}, 2=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>4, "pushpull"=>1, "ppname"=>"push", "note"=>"A1"}, 3=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>3, "pushpull"=>1, "ppname"=>"push", "note"=>"C#2"}, 4=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>2, "pushpull"=>1, "ppname"=>"push", "note"=>"A2"}, 5=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>1, "pushpull"=>1, "ppname"=>"push", "note"=>"G#2"}, 6=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>5, "pushpull"=>1, "ppname"=>"push", "note"=>"C1"}, 7=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>4, "pushpull"=>1, "ppname"=>"push", "note"=>"G1"}, 8=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>3, "pushpull"=>1, "ppname"=>"push", "note"=>"C2"}, 9=>{"side"=>1, "siden

# The numbers of the buttons
$tinakeys[15] is now:: 
{
"side"=>1, 
"sidename"=>"left", 
"row"=>3, 
"rowname"=>"g", 
"button"=>1, 
"pushpull"=>1, 
"ppname"=>"push", 
"note"=>"D3"
}


These numbers are arbitrary:
 number 1 is Left Side at top row, outer button, push.
 1..15 are all buttons on the left site, starting top row, outer button
 16..30 are the same button, but pulled.
 
 31..45 and 46..60 are the buttons on the right side, 
 starting with push again.
 

So $tinakeys[15] is the D3 
* on the left side (side=2, sidename=left)
* on the bottom row (rowname g, row 3 in this script), 
* inner button which is for finger 1, 
* on push (which is also pushpull=2 in this script). 


## $tinamap = onscreen names of buttons

In [35]:
$tinamap= Hash.new

$tinakeys.each {|h,k|
  $tinamap[h]=  k["sidename"][0].upcase+ k["rowname"][0].upcase+ k["button"].to_s+"-"+ k["ppname"].upcase+"="+ k["note"].upcase
    
  #puts $tinamap[h]
  }




{1=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>5, "pushpull"=>1, "ppname"=>"push", "note"=>"E1"}, 2=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>4, "pushpull"=>1, "ppname"=>"push", "note"=>"A1"}, 3=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>3, "pushpull"=>1, "ppname"=>"push", "note"=>"C#2"}, 4=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>2, "pushpull"=>1, "ppname"=>"push", "note"=>"A2"}, 5=>{"side"=>1, "sidename"=>"left", "row"=>1, "rowname"=>"t", "button"=>1, "pushpull"=>1, "ppname"=>"push", "note"=>"G#2"}, 6=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>5, "pushpull"=>1, "ppname"=>"push", "note"=>"C1"}, 7=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>4, "pushpull"=>1, "ppname"=>"push", "note"=>"G1"}, 8=>{"side"=>1, "sidename"=>"left", "row"=>2, "rowname"=>"c", "button"=>3, "pushpull"=>1, "ppname"=>"push", "note"=>"C2"}, 9=>{"side"=>1, "siden

# Now calculate distance map from button to button

## Distance from button to button: 
### the lower the value in distance, the easier the play from button -> button

### We calculate the score by comparing the two buttons.
* whether they are on the same side of the tina --> sameside(f,t)
* DELETED:: --whether they are on the same row --> samerow(f,t) ::DELETED
* whether you switch from Push to Pull --> ppchng(f,t)

### or by the attributes of the next button: 
* which row (top row, c row, g row) --> prefrow(f,t)
* preferred finger is 1, or 2. This will play on the inner buttons if possible --> preffing(f,t)

### hopping
* samefinger, same side, no switch from push to pull => hopping. Severe penalty --> hopp(f,t)



In [36]:
def sameside(f,t)
  score = 0
  if $tinakeys[f]["side"]==$tinakeys[t]["side"] 
    score = -1
  end
  result = score
end

def samerow(f,t)   # no in use anymore
  score = 0
  if $tinakeys[f]["row"]==$tinakeys[t]["row"] 
    score = 0
  end
  result = score
end

def hopp(f,t)
  score = 0
  
  # hopping defines as:
  # same finger has to jump from one button to another
  if (($tinakeys[f]["button"] == $tinakeys[t]["button"]) && 
      ($tinakeys[f]["side"] == $tinakeys[t]["side"])   )
    score = 100
  end
    
  #but... just same button push-pull, that's good, that's no hopping!
  if (($tinakeys[f]["row"]     == $tinakeys[t]["row"]) && 
      ($tinakeys[f]["side"]    == $tinakeys[t]["side"]) &&
      ($tinakeys[f]["button"]  == $tinakeys[t]["button"]) &&    
      ($tinakeys[f]["pushpull"]!= $tinakeys[t]["pushpull"]) )           
      
     score = -12  #-10
  end
  
  result = score
end

def prefrow(f,t)  
  # score = 15   # was +7, later +11. Penalty for the non c/g row
  if [2,3].include? $tinakeys[t]["row"]  # prefer c or g row
    score = 0
  else
    score = 15
  end
  result = score
end

def prefbuttt(t)
  score = 0
  if ($tinakeys[t]["button"]==1)  # prefer button one
    score = -4
  elsif
    ($tinakeys[t]["button"]==2)  # or prefer button two
    score = -2
  elsif
    ($tinakeys[t]["button"]==5)  # no button 5 please
    score = 5
  end  
  
  result = score
end

def ppchng(f,t)
  score = 2
  if $tinakeys[f]["pushpull"]==$tinakeys[t]["pushpull"] #push after push = good
    score = 0
  end
  result = score
end



:ppchng

In [37]:
# create a distance matrix, init with zeros ######################################
$distmat = Array.new(61){Array.new(61,0)}

#now loop over all 60 buttons
for f in 1..60 
  for t in 1..60

      if (f == t)    # same note? perfect, same button!! 
        score = -25  
      elsif (hopp(f,t) == 100)  # hop? no!
        score = 100
      else
        score = sameside(f,t)+
                #samerow(f,t)+
                hopp(f,t)    +
                prefrow(f,t) +
                prefbuttt(t) +
                ppchng(f,t) 
      end
      $distmat[f][t] = score
    
  end
end


1..60

# now the note -> buttons list
## for example: note B3 is on two positions: 42 and 55
## Make a list of this: note "B3" is at [42,55]. 

In [38]:
nlist=[]
for i in 1..60 do
  nlist << $tinakeys[i]["note"]
end
nlist= nlist.uniq.sort


#init to make it available outside the loop
$note_keys = Hash.new( [1] ) 

nlist.each { |nownote|
  $note_keys[nownote]=[]
  }

for i in 1..60 do
  $note_keys[$tinakeys[i]["note"]] << i
end
  
$note_keys

{"A1"=>[2, 26], "A2"=>[4, 25, 28], "A3"=>[54, 57], "A4"=>[35], "B1"=>[11, 22], "B2"=>[14, 51], "B3"=>[42, 55], "BB1"=>[17], "BB2"=>[20], "BB3"=>[49], "C#2"=>[3], "C#3"=>[31, 32, 46], "C#4"=>[34], "C1"=>[6], "C2"=>[8], "C3"=>[29, 36], "C4"=>[39, 58], "D#2"=>[18], "D#3"=>[47], "D#4"=>[45], "D2"=>[12, 23], "D3"=>[15, 52], "D4"=>[43, 50], "E1"=>[1], "E2"=>[9], "E3"=>[30, 37], "E4"=>[40, 59], "F#2"=>[27], "F#3"=>[56], "F#4"=>[60], "F1"=>[16], "F2"=>[24], "F3"=>[53], "G#2"=>[5], "G#3"=>[33], "G1"=>[7, 21], "G2"=>[10, 13, 19], "G3"=>[38, 41, 48], "G4"=>[44]}

# now it's time to do a serious amount of work

### calculate score of all combinations 
### if the list of possible routes is getting too long, clean it up 

In [39]:
# a = for example [[32, 54, 57], [42, 55], [39, 58], [56]]
# a is in this case a four note-tune 
a=[]
tune.each { |note|
  a << $note_keys[note]
}
 
pr=1
a.cycle(1) { |ar| pr = pr*ar.length
  }
puts "There are "+pr.to_s+" combinations for this tune."

if $dupchek  
  # duplicate check: play same note twice?  Skip one! 
  a_new=[]
  for i in 0..a.length-2
    a_new << a[i] unless a[i] == a[i+1]
  end

  a = a_new
  pr = 1
  a.cycle(1) { |ar| pr = pr*ar.length
    }
  puts "Cleaned up! Now there are "+pr.to_s+" combinations."
end


There are 61886548790943213277031694336 combinations for this tune.
Cleaned up! Now there are 254677155518284828300541952 combinations.


In [40]:
a

[[12, 23], [10, 13, 19], [4, 25, 28], [14, 51], [4, 25, 28], [10, 13, 19], [27], [10, 13, 19], [14, 51], [15, 52], [30, 37], [56], [38, 41, 48], [15, 52], [38, 41, 48], [29, 36], [38, 41, 48], [15, 52], [38, 41, 48], [56], [30, 37], [56], [15, 52], [30, 37], [56], [38, 41, 48], [15, 52], [29, 36], [14, 51], [4, 25, 28], [10, 13, 19], [12, 23], [8], [11, 22], [12, 23], [9], [8], [9], [10, 13, 19], [14, 51], [4, 25, 28], [10, 13, 19], [27], [10, 13, 19], [4, 25, 28], [14, 51], [29, 36], [14, 51], [4, 25, 28], [10, 13, 19], [12, 23], [27], [4, 25, 28], [15, 52], [56], [15, 52], [30, 37], [56], [38, 41, 48], [15, 52], [29, 36], [14, 51], [4, 25, 28], [10, 13, 19], [14, 51], [4, 25, 28], [10, 13, 19], [4, 25, 28], [14, 51], [29, 36], [14, 51], [4, 25, 28], [10, 13, 19], [27], [4, 25, 28], [15, 52], [56], [54, 57], [38, 41, 48], [56], [38, 41, 48], [15, 52], [29, 36], [14, 51], [4, 25, 28]]

In [41]:
  # This Function calculates the score of route (array of button numbers) 
  def calcscore(arr)   
        score = startbutton(arr[0].to_i)   # the startbutton
        for i in 0..(arr.length-2)
          # fro (i)   is the note you come from,
          # too (i+1) is the note to go to
          score = score+$distmat[arr[i].to_i][arr[i+1].to_i]
        end
        result = score.to_i
    end #~ def    

:calcscore

In [42]:
    # the first note of the tune has its own score
    def startbutton(f)
      score = 0
      if ($tinakeys[f]["button"] == 1)  # prefer button one
        score = -5
      elsif
         ($tinakeys[f]["button"] == 2)  # prefer button two as well
         score = -4
      elsif
         ($tinakeys[f]["button"] == 4)  # no button 4 please
         score = 3
      elsif
         ($tinakeys[f]["button"] == 5)  # no button 5 please
         score = 5
      end  

      if [2,3].include? $tinakeys[f]["row"]   # prefer c or g row
        score = score-5
      end
      result = score
    end

    #######################################################
    # VARIABLES with influence on solving the puzzle
    #  
    # if this script doesn't find a proper solution, 
    # raise this value to 120. Default is 30.
    uppervalscore = 30
    #
    # if there's no solution, larger the sample
    # default 2.0
    $divv = 2.0
    #######################################################


# first note in tune needs initialization:
nowatlength = 1
verz = []

# we start here!
b = a.shift  # this gets most left value of array in b, leave the rest in a
b.each { |e|
  verz << [e]
}
# Now verz has 1, 2 of 3 buttons where the first note of the tune is.

puts "Initiated! Now "+a.length.to_s+" notes left to do."
starttime = Time.now

#~ now loop the array by shifting it until zero length
while a.any? do
  
  
  #test
  divv = $divv
  #/test
  
  b = a.shift  
  
  puts "Now looking for "+b.join("-")+"\n"
  
  verz2 = verz.dup

  verz2.each_index { |v|    
    b.cycle(1) { |be|   
        
        #~ do not add notes on a high distance (uppervalscore)
        if $distmat[verz2[v].last][be] < uppervalscore
          verz << [verz2[v],be].flatten(1)  
        else 
          verz.delete(verz2[v])
        end
      }
  } #verz2
    
  #~  now remove the shorter arrays
  #~  todo: adding might be faster than deleting <- it's not. tested 2-12-2018
  nowatlength = nowatlength.next
  verz.delete_if { |v| v.flatten(1).length<nowatlength }
  
  puts "Continue with "+verz.length.to_s+" routes at "+a.length.to_s+" notes to go."
  
  if (verz.length==0)
    raise("No routes found. Raise uppervalscore to higher value (default=30, suggested 120).")
  end

  #  
  # cleanup the array by deleting the worst routes
  #
  if $cleanup && a.length>0
    
    verbose = true #### see on the screen what's going on #############
  
    while (verz.length > $Maxroutes)     
      # lookup the scores, delete all scores that are below '20%' value
      
      prevlength= verz.length.to_s
      
      #calculate scores
      tempscore=Array.new()
      scores=[]

      verz.each_index { |ec|
        e=verz[ec]
        
        sum=calcscore(e)
        
        tempscore[ec] = sum
        scores << sum
        
      } #verz
      

      scores.sort!      
      # puts("Got ",scores.length, " scores.") if verbose 
      
      val20 = scores[((scores.length)/divv).round.to_i]  # these are the sorted scores
      min = scores.min
      max = scores.max
      puts " - Score range: "+min.to_s+"-"+max.to_s+", "+(100/divv).to_i.to_s+"% at "+val20.to_s if verbose 
      
      nwverz = []
      verz.each_index { |ec|
        if calcscore(verz[ec]) <= val20
          e = verz[ec]
          nwverz << e
        end
        }
      
      
      verz = nwverz.dup      ##########TODO
      
      #~ delete if score < 20% value
      #~ 
      puts " - Array shrunk from "+prevlength+" to "+verz.length.to_s+" values." if verbose
      
      if prevlength==verz.length.to_s
        puts(" - No results, adjusting parameter to ", divv)
        divv = divv + 0.10
      end #if
      
    end #(verz.length)  
  end  #cleanup
end  # while

puts starttime
puts Time.now


Initiated! Now 84 notes left to do.
Now looking for 10-13-19

Continue with 5 routes at 83 notes to go.
Now looking for 4-25-28

Continue with 15 routes at 82 notes to go.
Now looking for 14-51

Continue with 25 routes at 81 notes to go.
Now looking for 4-25-28

Continue with 65 routes at 80 notes to go.
Now looking for 10-13-19

Continue with 195 routes at 79 notes to go.
Now looking for 27

Continue with 195 routes at 78 notes to go.
Now looking for 10-13-19

Continue with 585 routes at 77 notes to go.
Now looking for 14-51

Continue with 975 routes at 76 notes to go.
Now looking for 15-52

Continue with 1950 routes at 75 notes to go.
Now looking for 30-37

Continue with 3900 routes at 74 notes to go.
Now looking for 56

Continue with 3900 routes at 73 notes to go.
Now looking for 38-41-48

Continue with 11700 routes at 72 notes to go.
 - Score range: -90-43, 50% at -18
 - Array shrunk from 11700 to 6011 values.
 - Score range: -90--18, 50% at -33
 - Array shrunk from 6011 to 3063 va

Now looking for 4-25-28

Continue with 7732 routes at 22 notes to go.
 - Score range: -407--378, 50% at -394
 - Array shrunk from 7732 to 4542 values.
 - Score range: -407--394, 50% at -397
 - Array shrunk from 4542 to 2576 values.
Now looking for 10-13-19

Continue with 7728 routes at 21 notes to go.
 - Score range: -422--385, 50% at -398
 - Array shrunk from 7728 to 3942 values.
Now looking for 14-51

Continue with 7884 routes at 20 notes to go.
 - Score range: -425--400, 50% at -415
 - Array shrunk from 7884 to 3994 values.
Now looking for 4-25-28

Continue with 9734 routes at 19 notes to go.
 - Score range: -428--400, 50% at -417
 - Array shrunk from 9734 to 5130 values.
 - Score range: -428--417, 50% at -419
 - Array shrunk from 5130 to 3884 values.
Now looking for 10-13-19

Continue with 11652 routes at 18 notes to go.
 - Score range: -443--407, 50% at -419
 - Array shrunk from 11652 to 6312 values.
 - Score range: -443--419, 50% at -434
 - Array shrunk from 6312 to 3516 values.


In [43]:
$routes=Array.new()

#init 
min=65535
scores=[]

verz.each_index { |ec|
  e=verz[ec]
  sum=calcscore(e)
  
  $routes[ec] = sum
  scores << sum  
} #verz

min = scores.min
max  = 0
nmax = 0

for i in min...(min+100) do
  puts "Score "+i.to_s+": "+scores.count(i).to_s  if scores.count(i)>0
  max  = max+scores.count(i)
  nmax = i if max <= 20    # if more than 20 routes, stop adding to bestroutes
end

puts "Now list all scores that are <= "+ nmax.to_s



Score -585: 8
Score -584: 16
Score -583: 32
Score -582: 48
Score -581: 160
Score -580: 280
Score -579: 536
Score -578: 808
Score -577: 1732
Score -576: 328
Score -575: 680
Score -574: 1056
Score -573: 1580
Score -572: 64
Score -571: 184
Score -570: 296
Score -567: 8
Score -566: 8
Score -565: 16
Score -564: 32
Score -563: 48
Score -562: 152
Score -561: 264
Score -560: 496
Score -559: 744
Score -558: 1540
Now list all scores that are <= -585


# best routes

In [44]:
$bestroutes=[] 

verz.each_index{ |ec|
  e = verz[ec]
  if $routes[ec] <= nmax    
    $bestroutes << e
  end  
}

puts "Ready! I have found "+$bestroutes.length.to_s+" paths that have score <= "+nmax.to_s



Ready! I have found 8 paths that have score <= -585


##  Variable   min   contains the lowest score, which is the best path 
##  nmax   is the value with about the best ~20 paths.
##  $bestroutes     is an array containing the best paths. 

In [45]:
# now who is the winner?
winner=[]

$bestroutes.each { |oneroute|
  winner << oneroute if calcscore(oneroute)==min
  }


if false
  sscores=[]
  $bestroutes.each { |oneroute|
    sscores <<  calcscore(oneroute)
    }

  winner=[]
  mmin=scores.min
  $bestroutes.each { |oneroute|
    winner << oneroute if calcscore(oneroute)==mmin
    }
end


# this is the main result of the script

In [46]:
# now we want to now a little more about what we found.
# so, from the best routes
# make a 'median' route


tunelength = $bestroutes[0].length 
nrofbest = $bestroutes.length

puts "these " + nrofbest.to_s + " are the best routes in :"
for i in 0..tunelength-1
  bestnotes = []
  $bestroutes.each_index { |route|
    bestnotes << $bestroutes[route][i]
  }
  
  min=bestnotes.min
  max=bestnotes.max
  
  # Pretty sure about a button when  min==max
  # else there is no obvious winner 
  print "OK:" if min==max
  print "??:" if min!=max
  
  puts bestnotes.group_by { |e| e }.map{ |k, v| [k, v.length] }.to_h.to_s + 
       bestnotes.group_by { |e| e }.map{ |k, v| [k, $tinamap[k]] }.to_h.to_s
    
end 


these 8 are the best routes in :
OK:{23=>8}{23=>"LC3-PULL=D2"}
OK:{10=>8}{10=>"LC1-PUSH=G2"}
OK:{25=>8}{25=>"LC1-PULL=A2"}
OK:{51=>8}{51=>"RC1-PULL=B2"}
OK:{25=>8}{25=>"LC1-PULL=A2"}
OK:{10=>8}{10=>"LC1-PUSH=G2"}
OK:{27=>8}{27=>"LG4-PULL=F#2"}
OK:{10=>8}{10=>"LC1-PUSH=G2"}
OK:{14=>8}{14=>"LG2-PUSH=B2"}
OK:{15=>8}{15=>"LG1-PUSH=D3"}
OK:{30=>8}{30=>"LG1-PULL=E3"}
OK:{56=>8}{56=>"RG1-PULL=F#3"}
OK:{41=>8}{41=>"RG1-PUSH=G3"}
OK:{15=>8}{15=>"LG1-PUSH=D3"}
??:{38=>4, 41=>4}{38=>"RC3-PUSH=G3", 41=>"RG1-PUSH=G3"}
??:{36=>4, 29=>4}{36=>"RC1-PUSH=C3", 29=>"LG2-PULL=C3"}
??:{38=>4, 41=>4}{38=>"RC3-PUSH=G3", 41=>"RG1-PUSH=G3"}
OK:{15=>8}{15=>"LG1-PUSH=D3"}
OK:{41=>8}{41=>"RG1-PUSH=G3"}
OK:{56=>8}{56=>"RG1-PULL=F#3"}
OK:{30=>8}{30=>"LG1-PULL=E3"}
OK:{56=>8}{56=>"RG1-PULL=F#3"}
OK:{15=>8}{15=>"LG1-PUSH=D3"}
OK:{30=>8}{30=>"LG1-PULL=E3"}
OK:{56=>8}{56=>"RG1-PULL=F#3"}
OK:{41=>8}{41=>"RG1-PUSH=G3"}
OK:{15=>8}{15=>"LG1-PUSH=D3"}
OK:{36=>8}{36=>"RC1-PUSH=C3"}
OK:{51=>8}{51=>"RC1-PULL=B2"}
OK:{25=>8}{25=

0..84

In [47]:
#
# more details on the
# winner(s) route(s)
# 

puts Time.now

if winner.length<20

  winner.each { |e|
    sum = calcscore(e)
    puts "route:" + e.join(",") + ", score: " + sum.to_s

    e.each_index { |ee|
      fro = e[ee-1]
      too = e[ee]
      line= too.to_s.rjust(2,"0") + "; " + $tinamap[too].to_s + ";"  

      line.include?("#") ? line=line+" " : line=line+"  " # add aspace if no  sharp#
      line = line + fro.to_s.rjust(2,"0") + "->" + too.to_s.rjust(2,"0") + "; " + $distmat[fro][too].to_s.rjust(4," ") + ";" if ee>0
      line = line + " + "   if $distmat[fro][too]>0    # add a '+' 
      puts line
      }
    }
else
  puts "Too much winners ("+ winner.length.to_s+") to list then all!"
end


2018-12-28 13:26:08 +0100
route:23,10,25,51,25,10,27,10,14,15,30,56,41,15,38,36,38,15,41,56,30,56,15,30,56,41,15,36,51,25,10,23,8,22,23,9,8,9,10,14,25,10,27,10,25,51,36,51,25,10,12,27,25,52,56,15,30,56,41,15,36,51,25,10,14,25,10,25,51,36,51,25,10,27,25,52,56,57,41,56,41,15,36,51,25, score: -585
23; LC3-PULL=D2;  
10; LC1-PUSH=G2;  23->10;   -3;
25; LC1-PULL=A2;  10->25;  -15;
51; RC1-PULL=B2;  25->51;   -4;
25; LC1-PULL=A2;  51->25;   -4;
10; LC1-PUSH=G2;  25->10;  -15;
27; LG4-PULL=F#2; 10->27;    1; + 
10; LC1-PUSH=G2;  27->10;   -3;
14; LG2-PUSH=B2;  10->14;   -3;
15; LG1-PUSH=D3;  14->15;   -5;
30; LG1-PULL=E3;  15->30;  -15;
56; RG1-PULL=F#3; 30->56;   -4;
41; RG1-PUSH=G3;  56->41;  -15;
15; LG1-PUSH=D3;  41->15;   -4;
38; RC3-PUSH=G3;  15->38;    0;
36; RC1-PUSH=C3;  38->36;   -5;
38; RC3-PUSH=G3;  36->38;   -1;
15; LG1-PUSH=D3;  38->15;   -4;
41; RG1-PUSH=G3;  15->41;   -4;
56; RG1-PULL=F#3; 41->56;  -15;
30; LG1-PULL=E3;  56->30;   -4;
56; RG1-PULL=F#3; 30->56;   -4;
15; LG1-PU

36; RC1-PUSH=C3;  15->36;   -4;
51; RC1-PULL=B2;  36->51;  -15;
25; LC1-PULL=A2;  51->25;   -4;
10; LC1-PUSH=G2;  25->10;  -15;
14; LG2-PUSH=B2;  10->14;   -3;
25; LC1-PULL=A2;  14->25;   -3;
10; LC1-PUSH=G2;  25->10;  -15;
25; LC1-PULL=A2;  10->25;  -15;
51; RC1-PULL=B2;  25->51;   -4;
36; RC1-PUSH=C3;  51->36;  -15;
51; RC1-PULL=B2;  36->51;  -15;
25; LC1-PULL=A2;  51->25;   -4;
10; LC1-PUSH=G2;  25->10;  -15;
27; LG4-PULL=F#2; 10->27;    1; + 
25; LC1-PULL=A2;  27->25;   -5;
52; RC2-PULL=D3;  25->52;   -2;
56; RG1-PULL=F#3; 52->56;   -5;
57; RG2-PULL=A3;  56->57;   -3;
41; RG1-PUSH=G3;  57->41;   -3;
56; RG1-PULL=F#3; 41->56;  -15;
41; RG1-PUSH=G3;  56->41;  -15;
15; LG1-PUSH=D3;  41->15;   -4;
36; RC1-PUSH=C3;  15->36;   -4;
51; RC1-PULL=B2;  36->51;  -15;
25; LC1-PULL=A2;  51->25;   -4;
route:23,10,25,51,25,10,27,10,14,15,30,56,41,15,38,36,38,15,41,56,30,56,15,30,56,41,15,36,51,25,10,23,8,22,23,9,8,9,10,51,25,10,27,10,25,51,36,51,25,10,12,27,25,52,56,15,30,56,41,15,36,51,25,10,51,

08; LC3-PUSH=C2;  09->08;   -1;
09; LC2-PUSH=E2;  08->09;   -3;
10; LC1-PUSH=G2;  09->10;   -5;
14; LG2-PUSH=B2;  10->14;   -3;
25; LC1-PULL=A2;  14->25;   -3;
10; LC1-PUSH=G2;  25->10;  -15;
27; LG4-PULL=F#2; 10->27;    1; + 
10; LC1-PUSH=G2;  27->10;   -3;
25; LC1-PULL=A2;  10->25;  -15;
51; RC1-PULL=B2;  25->51;   -4;
36; RC1-PUSH=C3;  51->36;  -15;
51; RC1-PULL=B2;  36->51;  -15;
25; LC1-PULL=A2;  51->25;   -4;
10; LC1-PUSH=G2;  25->10;  -15;
12; LG4-PUSH=D2;  10->12;   -1;
27; LG4-PULL=F#2; 12->27;  -11;
25; LC1-PULL=A2;  27->25;   -5;
52; RC2-PULL=D3;  25->52;   -2;
56; RG1-PULL=F#3; 52->56;   -5;
15; LG1-PUSH=D3;  56->15;   -2;
30; LG1-PULL=E3;  15->30;  -15;
56; RG1-PULL=F#3; 30->56;   -4;
41; RG1-PUSH=G3;  56->41;  -15;
15; LG1-PUSH=D3;  41->15;   -4;
36; RC1-PUSH=C3;  15->36;   -4;
51; RC1-PULL=B2;  36->51;  -15;
25; LC1-PULL=A2;  51->25;   -4;
10; LC1-PUSH=G2;  25->10;  -15;
51; RC1-PULL=B2;  10->51;   -2;
25; LC1-PULL=A2;  51->25;   -4;
10; LC1-PUSH=G2;  25->10;  -15;
25; L

[[23, 10, 25, 51, 25, 10, 27, 10, 14, 15, 30, 56, 41, 15, 38, 36, 38, 15, 41, 56, 30, 56, 15, 30, 56, 41, 15, 36, 51, 25, 10, 23, 8, 22, 23, 9, 8, 9, 10, 14, 25, 10, 27, 10, 25, 51, 36, 51, 25, 10, 12, 27, 25, 52, 56, 15, 30, 56, 41, 15, 36, 51, 25, 10, 14, 25, 10, 25, 51, 36, 51, 25, 10, 27, 25, 52, 56, 57, 41, 56, 41, 15, 36, 51, 25], [23, 10, 25, 51, 25, 10, 27, 10, 14, 15, 30, 56, 41, 15, 38, 36, 38, 15, 41, 56, 30, 56, 15, 30, 56, 41, 15, 36, 51, 25, 10, 23, 8, 22, 23, 9, 8, 9, 10, 14, 25, 10, 27, 10, 25, 51, 36, 51, 25, 10, 12, 27, 25, 52, 56, 15, 30, 56, 41, 15, 36, 51, 25, 10, 51, 25, 10, 25, 51, 36, 51, 25, 10, 27, 25, 52, 56, 57, 41, 56, 41, 15, 36, 51, 25], [23, 10, 25, 51, 25, 10, 27, 10, 14, 15, 30, 56, 41, 15, 38, 36, 38, 15, 41, 56, 30, 56, 15, 30, 56, 41, 15, 36, 51, 25, 10, 23, 8, 22, 23, 9, 8, 9, 10, 51, 25, 10, 27, 10, 25, 51, 36, 51, 25, 10, 12, 27, 25, 52, 56, 15, 30, 56, 41, 15, 36, 51, 25, 10, 14, 25, 10, 25, 51, 36, 51, 25, 10, 27, 25, 52, 56, 57, 41, 56, 41, 15

In [48]:
# end of script
# concertina-6.rb